In [1]:
import os
import openai
import panel as pn  # GUI
import re  # For input validation
from dotenv import load_dotenv, find_dotenv
from geopy.geocoders import Nominatim
from geopy.distance import geodesic

# Load environment variables
_ = load_dotenv(find_dotenv())  

# OpenAI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Initialize geocoder
geolocator = Nominatim(user_agent="nitti_bot")

# Predefined Store Locations with Coordinates
stores = {
    "H K ENGINEERING PTE LTD - 49 Joo Koon Cir": (1.3273, 103.6730),
    "KOON HIN LEE SAFETY PRODUCTS - 221 Boon Lay Place, Boon Lay Shopping Centre": (1.3474, 103.7138),
    "TAKA HARDWARE & ENGINEERING PTE LTD - 32 Lok Yang Way": (1.3216, 103.6851),
    "SINYOU HARDWARE PTE LTD - 5 Soon Lee St, Pioneer Point": (1.3275, 103.6961),
    "MUI HUAT TRADING - 221 Boon Lay Place, Boon Lay Shopping Centre": (1.3474, 103.7138),
    "NS HARDWARE PTE LTD - 2 Buroh Cres, Ace@Buroh": (1.3262, 103.6987),
    "AGGREGATES ENGINEERING PTE LTD - 2B Tuas Ave 12": (1.3221, 103.6525),
    "SIN HILL INTERNATIONAL PTE LTD - 11 Woodlands Close": (1.4289, 103.7954),
    "ISSA LEATHER BAGS AND SHOES SHOP - 4 Woodlands Street 12, Marsiling Mall": (1.4337, 103.7741),
    "KIAN HUAT ENTERPRISES - 39 Woodlands Cl, Mega@Woodlands": (1.4305, 103.7852),
    "UNIQUE HARDWARE PTE LTD - 30 Kranji Loop, TimMac@Kranji": (1.4267, 103.7623),
    "JAHO TRADING PTE LTD - 60 Jalan Lam Huat, Carros Centre": (1.4316, 103.7859),
    "KIANG SING HONG PTE LTD - 280 Woodlands Industrial Park E5, Harvest Building": (1.4472, 103.7920),
    "KIAW AIK HARDWARE TRADING PTE LTD - 321 Jln Besar": (1.3098, 103.8571),
    "MQ HARDWARE & ELECTRICAL TRADING PTE LTD - 3 Toa Payoh Industrial Park": (1.3323, 103.8518),
    "NIRJA MINI MART PTE.LTD - 66 Desker Road": (1.3077, 103.8559),
    "HAN SIANG HARDWARE PTE LTD - 30 Kelantan Rd": (1.3056, 103.8583),
    "TROSEAL BUILDING MATERIALS PTE LTD - 637 Veerasamy Rd": (1.3065, 103.8568),
    "YIAP HENG CHEONG HARDWARE PTE LTD - 294 Balestier Rd": (1.3204, 103.8525),
    "FRANCE SHOES COMPANY PTE LTD - 1 Park Road, People’s Park Complex": (1.2852, 103.8442),
    "WANTA GLOTRADE PTE LTD - 1 Park Road, People’s Park Complex": (1.2852, 103.8442),
    "TECK MENG HARDWARE PTE LTD - 5030 Ang Mo Kio Ind Park 2": (1.3687, 103.8602),
    "FASTENER GROUP PTE LTD - 8 Kaki Bukit Ave 4, Premier @Kaki Bukit": (1.3367, 103.9067),
    "LEONG SENG INDUSTRIAL PTE LTD - 5070 Ang Mo Kio Ind Park 2": (1.3693, 103.8591),
    "CHANG TAI CHIANG HARDWARE PTE LTD - 5071 Ang Mo Kio Ind Park 2": (1.4013, 103.8158),
    "NAM YONG INDUSTRADES - 10 Kaki Bukit Rd 2, First East Centre": (1.3359, 103.9058),
    "HORME HARDWARE - 1 Ubi Crescent, Number One Building": (1.3299, 103.8944),
    "HORME HARDWARE - 341 Changi Road": (1.3223, 103.9063),
    "MENG TAT HARDWARE CO - 6 Ubi Road 1, Wintech Centre": (1.3270, 103.8980),
    "CHIN HOE HUP KEE HARDWARE - 139 Tampines St 11": (1.3457, 103.9443),
    "LI FONG HARDWARE ENTERPRISE - 3018 Bedok North Street 5, EastLink": (1.3341, 103.9531),
    "B&S (2017) HARDWARE PTE LTD - 308 Geylang Road": (1.3137, 103.8804),
    "J&E TRADING PTE LTD - 117 Upper East Coast Road": (1.3125, 103.9228),
    "AGGREGATES ENGINEERING PTE LTD - 3024 Ubi Road 3, Kampong Ubi Industrial Estate": (1.3302, 103.8981),
}

# Function to get coordinates from postal code dynamically
def get_coordinates(postal_code):
    try:
        location = geolocator.geocode(postal_code + ", Singapore")
        if location:
            return (location.latitude, location.longitude)
        else:
            return None
    except Exception as e:
        print("Error fetching coordinates:", e)
        return None

# Function to validate postal code (Singapore 6-digit format)
def validate_postal(postal_code):
    return re.match(r'^\d{6}$', postal_code) is not None

# Function to find the nearest store (Only for Singapore)
def find_nearest_store(postal_code):
    if not validate_postal(postal_code):
        return "Invalid postal code format. Please enter a 6-digit Singapore postal code."
    
    user_coords = get_coordinates(postal_code)
    if not user_coords:
        return "Could not find location for this postal code."

    nearest_store = min(stores.items(), key=lambda store: geodesic(user_coords, store[1]).kilometers)
    
    return f"Nearest store: {nearest_store[0]} (Distance: {geodesic(user_coords, nearest_store[1]).kilometers:.2f} km)"

# Function to validate email
def is_valid_email(email):
    return re.match(r"[^@]+@[^@]+\.[^@]+", email)

# Function to validate phone number (accepts international format)
def is_valid_phone(phone):
    return re.match(r"^\+?\d{10,15}$", phone)

# Function to communicate with OpenAI
def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  # Controls randomness
    )
    return response.choices[0].message["content"]

# Define chatbot context
context = [ {'role':'system', 'content':"""
You are CustomerserviceBot for Nitti Safety Footwear, an automated service to assist incoming enquiries.
You first greet the customer, then assist with the enquiry regarding safety shoes,
and then ask if our sales can reach out to them.
If you do not know the answer, ask if it is okay that Mr. Harmsen contacts them.
You wait to collect all the information, then summarize it and check for a final time if the customer needs anything else.
Finally, request their phone number and email.
When asking for a phone number, always check the country code.
Ensure clarity on product models, accessories, and customer details.
You respond in a short, friendly, and conversational style, without repeating questions more than twice.
Orders cannot be placed via this chatbot; instead, collect the user's contact details for a sales follow-up.

The list includes:
Low cut models:
- 21281, black, Rating: CE EN 20345:2011 S1P SRC, Model: Lace, Size: 2-14 (UK) / 35-48 (EU), Upper: Leather, Lining: Genuine Cambrelle®, 
  Outsole: Direct Injection Polyurethane
- 21381, Rating: CE EN 20345:2011 S1P SRC, Type: Low Cut, Model: Velcro & Ventilated, Size: 2-14 (UK) or 35-48 (EU), Color: Black or Brown, Upper: Leather,
  Lining: Cambrelle®, Outsole: Direct Injection Polyurethane
- 21981 black, Rating: CE EN 20345:2011 S1P SRC, Type: Low Cut, Model: Slip-on, Size: 2-14 (UK) or 35-48 (EU), Color: Black, Upper: Leather, Lining: Cambrelle®, Outsole: Direct Injection Polyurethane

Mid cut models:
- 22281 black, Rating: CE EN 20345:2011 S1P SRC, Type: Low Cut, Model: lace, Size: 2-14 (UK) or 35-48 (EU), Upper: Leather, Lining: Cambrelle®, Outsole: Direct Injection Polyurethane
- 22681 black, Rating: CE EN 20345:2011 S3 SRC, Type: Mid Cut, Model: Zipper, Size: 2-14 (UK) or 35-48 (EU), Upper: Water Resistant Leather, Lining: Cambrelle®, Outsole: Direct Injection Polyurethane
- 22781 black and brown, Rating: CE EN 20345:2011 S3 SRC, Type: Mid Cut, Model: slip on, Size: 2-14 (UK) or 35-48 (EU), Upper: Water Resistant Leather, Lining: Cambrelle®, Outsole: Direct Injection Polyurethane

High cut models:
- 23281 black and brown, CE EN 20345:2011 S3 SRC, Type: High Cut, Model: Pull-on, Size: 2-14 (UK) or 35-48 (EU), Upper: Water Resistant Leather, Lining: Cambrelle®, Outsole: Direct Injection Polyurethane
- 23681 black and brown, CE EN 20345:2011 S3 SRC, Type: High Cut, Model: zipper, Size: 2-14 (UK) or 35-48 (EU), Upper: Water Resistant Leather, Lining: Cambrelle®, Outsole: Direct Injection Polyurethane
- 23381 black and brown, CE EN 20345:2011 S3 SRC, Type: High Cut, Model: lace, Size: 2-14 (UK) or 35-48 (EU), Upper: Water Resistant Leather, Lining: Cambrelle®, Outsole: Direct Injection Polyurethane

Accessories:
- Socks
- Shoe Guard
- T-shirt
- Mouse pad

Information about the models you use during the conversation:
- Models to be used in wet and waterproof environments are the mid and high cut models, not the low cut.
- 21281: This model is designed to provide you with extra comfort during those long working hours.
- 21981: A general purpose and well ventilated model with an industrial grade Velcro strap for added functionality.
  Improved ventilation to maintain comfort for workers in hot environments.

Safety ratings:
- Indonesia Rating: SNI 7079-2009
- Singapore Rating: SS 513
- China Rating: GB 21148
- Malaysia Rating: Sirim
- Other: As/NZS 2210.3.2019 and OSHC

Information about direct injection to use:
Direct injection is the process where the PU is directly injected into the mold as opposed to glued to the upper of the shoe. 
This ensures a better bond and is of higher quality than non-direct injection.

Warranty information:
Nitti Safety Footwear is warranted for six (6) months from the date of invoice by Nitti to its distributors
against defects in materials and/or workmanship when used under normal conditions for its intended purpose.

Return policy:
As a requirement of the warranty mentioned above, the purchaser must return the footwear to Nitti for assessment together with proof of purchase or receipt.
Following assessment, if Nitti determines that the footwear is defective as the result of normal use, Nitti will replace the footwear.

Boot care:
1. Do not store your boots in direct sunlight. Always air-dry your boots after use and store them in a cool, dry, and well-ventilated place.
2. Clean and polish your boots regularly with wax to enhance durability. Be sure to rinse your boots with water after contact with cement. 
   Left unattended, cement will damage the leather upper, drying it out and causing cracks to form.
3. Do not wash your boots. Wet boots should be air-dried naturally at room temperature with the cushion footbed and laces removed and the boots fully opened. 
   Never force dry or use strong detergents or caustic cleaning agents.

Store information:
H K ENGINEERING PTE LTD, 49 Joo Koon Cir, Singapore 629068, Tel: 6565 5555
KOON HIN LEE SAFETY PRODUCTS, 221 Boon Lay Place #01-238, Boon Lay Shopping Centre,640221, Tel: 6268 0401
TAKA HARDWARE & ENGINEERING PTE LTD, 32 Lok Yang Way, Singapore 628639, TEL: 6842 0782
SINYOU HARDWARE PTE LTD, 5 Soon Lee St #01-21, Pioneer Point, Singapore 627607, Tel: 6710 5955
MUI HUAT TRADING, 221 Boon Lay Place #01-238, Boon Lay Shopping Centre, Singapore 640221, Tel: 6261 3590
NS HARDWARE PTE LTD, 2 Buroh Cres, #01-15, Ace@Buroh, Singapore 627546, Tel: 6029 3113
AGGREGATES ENGINEERING PTE LTD, 2B Tuas Ave 12, Singapore 639048, Tel: 6741 4638/ 69/ 72
SIN HILL INTERNATIONAL PTE LTD, 11 Woodlands Close #05-32, Singapore 737853, Tel: 6555 1658
ISSA LEATHER BAGS AND SHOES SHOP, 4 Woodlands Street 12, #02-49, Marsiling Mall, Singapore 738623, Tel: 6269 5178
KIAN HUAT ENTERPRISES, 39 Woodlands Cl, #01-13, Mega@Woodlands, Singapore 737856, Tel: 6779 9686
UNIQUE HARDWARE PTE LTD, 30 Kranji Loop BlkB, #04-17, TimMac@Kranji, 739570. Tel: 6748 4211
JAHO TRADING PTE LTD, 60 Jalan Lam Huat #01-16, Carros Centre, Singapore 737869, Tel: 6481 7305
KIANG SING HONG PTE LTD, 280 Woodlands Industrial Park E5 #01-45/46,, Harvest Building, Singapore 757322, Tel: 6760 0365
KIAW AIK HARDWARE TRADING PTE LTD, 321 Jln Besar, Singapore 208979, Tel: 6296 8858
MQ HARDWARE & ELECTRICAL TRADING PTE LTD, 3 Toa Payoh Industrial Park, #01-1361, Singapore 319055, Tel: 9665 2756
NIRJA MINI MART PTE.LTD, 66 Desker Road, Singapore 209589, Tel: 8182 3815
HAN SIANG HARDWARE PTE LTD, 30 Kelantan Rd, #01-83,, Singapore 200030, Tel: 6294 5395
TROSEAL BUILDING MATERIALS PTE LTD, 637 Veerasamy Rd, #01-123/125, Singapore 200637, Tel: 6298 1123
YIAP HENG CHEONG HARDWARE PTE LTD, 294 Balestier Rd, Singapore 3299735, Tel: 6254 4623
FRANCE SHOES COMPANY PTE LTD, 1 Park Road, #02-35, People’s Park Complex, Singapore 059108, Tel: 6535 0042
WANTA GLOTRADE PTE LTD, 1 Park Road #02-03, People’s Park Complex, Singapore 059108, Tel: 6535 0133/ 0624
TECK MENG HARDWARE PTE LTD, 5030 Ang Mo Kio Ind Park 2, #01-221, Singapore 569533, Tel: 9763 5706
FASTENER GROUP PTE LTD, 8 Kaki Bukit Ave 4, #03-20, Premier @Kaki Bukit, Singapore 415875, Tel: 6384 6355/ 6384 6913
LEONG SENG INDUSTRIAL PTE LTD, 5070 Ang Mo Kio Ind Park 2, #01-1491, Singapore 569567, Tel: 6483 2409
CHANG TAI CHIANG HARDWARE PTE LTD, 5071 Ang Mo Kio Ind Park 2, #01-1117, Singapore 787812, Tel: 6481 0231
NAM YONG INDUSTRADES, 10 Kaki Bukit Rd 2, #01-02, First East Centre, Singapore 417868, Tel: 6382 5465
HORME HARDWARE, 1 Ubi Crescent #01-01, Number One Building, Singapore 408563, Tel: 6840 8855
HORME HARDWARE, 341 Changi Road, Singapore 419812, Tel: 6840 8844
MENG TAT HARDWARE CO, 6 Ubi Road 1 #01-08/09, Wintech Centre, Singapore 408726, Tel: 6292 9484
CHIN HOE HUP KEE HARDWARE, 139 Tampines St 11, #01-26, Singapore 521139, Tel: 6785 1910
LI FONG HARDWARE ENTERPRISE, 3018 Bedok North Street 5, #01-18, EastLink, Singapore 486132, Tel: 6444 6231
B&S (2017) HARDWARE PTE LTD, 308 Geylang Road, Singapore 389348, Tel: 9616 6571
J&E TRADING PTE LTD, 117 Upper East Coast Road, #01-01, Singapore 455243, Tel: 8918 0458
AGGREGATES ENGINEERING PTE LTD, 3024 Ubi Road 3, #02-71, Kampong Ubi Industrial Estate, Singapore 408652, Tel: 6741 4638/ 69/ 72
"""} ]


# GUI Setup
pn.extension()
panels = []  # Collect display messages

# User Details Input
email_input = pn.widgets.TextInput(placeholder="Enter your email...")
phone_input = pn.widgets.TextInput(placeholder="Enter your phone number...")
country_dropdown = pn.widgets.Select(name="Select Country", options=["Singapore", "Malaysia", "Indonesia", "Thailand", "Vietnam", "Philippines", "China", "Other"])
postal_input = pn.widgets.TextInput(placeholder="Enter your Singapore postal code... (Required if in SG)")
submit_button = pn.widgets.Button(name="Submit Details", button_type="primary")

# Disable chatbot input until details are provided
inp = pn.widgets.TextInput(value="", placeholder='Enter text here…', disabled=True)
button_conversation = pn.widgets.Button(name="Chat!", disabled=True)

# Function to validate user details before starting chatbot
def validate_and_start(event=None):  
    email = email_input.value.strip()
    phone = phone_input.value.strip()
    country = country_dropdown.value
    postal = postal_input.value.strip()

    if not is_valid_email(email):
        return pn.pane.Markdown("❌ **Invalid email. Please enter a valid email.**", styles={'color': 'red'})

    if not is_valid_phone(phone):
        return pn.pane.Markdown("❌ **Invalid phone number. Please enter a valid number.**", styles={'color': 'red'})

    # Validate postal code only if country is Singapore
    if country == "Singapore":
        if not validate_postal(postal):
            return pn.pane.Markdown("❌ **Invalid postal code. Please enter a valid 6-digit Singapore postal code.**", styles={'color': 'red'})
        store = find_nearest_store(postal)
    else:
        postal = "N/A"
        store = None  # No store search for overseas users

    # Update chatbot context
    context.append({'role': 'system', 'content': f"User email: {email}, User phone: {phone}, User country: {country}, User postal code: {postal}, Nearest store: {store if store else 'N/A'}"})

    # Enable chatbot input
    inp.disabled = False
    button_conversation.disabled = False

    success_message = f"✅ **Details saved! {'Nearest store: ' + store if store else 'You can now chat.'}**"
    return pn.pane.Markdown(success_message, styles={'color': 'green'})

# Bind validation function to button
validate_form = pn.bind(validate_and_start, submit_button)

# Collect messages function
def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': prompt})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': response})
    panels.append(pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(pn.Row('Nitti CS:', pn.pane.Markdown(response, width=600, styles={'background-color': '#F6F6F6'})))
    return pn.Column(*panels)

interactive_conversation = pn.bind(collect_messages, button_conversation)

# Dashboard Layout
dashboard = pn.Column(
    "**📢 Please enter your contact details before starting the chat:**",
    email_input,
    phone_input,
    country_dropdown,
    postal_input,
    pn.Row(submit_button),
    pn.panel(validate_form, loading_indicator=True),
    pn.layout.Divider(),
    "**💬 Chat with the Nitti Safety Footwear Bot:**",
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] Markdown(str)
    [1] TextInput(placeholder='Enter your email...')
    [2] TextInput(placeholder='Enter your phone n...)
    [3] Select(options=['Singapore', ...], value='Singapore')
    [4] TextInput(placeholder='Enter your Singapore p...)
    [5] Row
        [0] Button(button_type='primary', name='Submit Details')
    [6] ParamFunction(function, _pane=Markdown, defer_load=False, loading_indicator=True)
    [7] Divider()
    [8] Markdown(str)
    [9] TextInput(disabled=True, placeholder='Enter text here…')
    [10] Row
        [0] Button(disabled=True, name='Chat!')
    [11] ParamFunction(function, _pane=Column, defer_load=False, height=300, loading_indicator=True)

In [2]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [6]:
!git clone https://github.com/radigu74/nitti-chatbot.git

Cloning into 'nitti-chatbot'...


In [9]:
python -m venv venv
venv\Scripts\activate

SyntaxError: invalid syntax (763650476.py, line 1)